## Data Loading

In [14]:
import pandas as pd
df = pd.read_csv('top_30_mutual_funds.csv',encoding='latin1')
df.head()

,scheme_name,min_sip,min_lumpsum,expense_ratio,fund_size_cr,fund_age_yr,fund_manager,sortino,alpha,sd,...,risk_level,amc_name,rating,category,sub_category,returns_1yr,returns_3yr,returns_5yr,composite_score,rank
0,Quant Small Cap Fund,1000,5000,0.64,3301,10,Sanjeev Sharma,3.71,19.16,24.75,...,6,Quant Mutual Fund,5,Equity,Small Cap Mutual Funds,5.4,71.4,23.2,0.667338,1
1,Quant Infrastructure Fund,1000,5000,0.64,822,10,Vasav Sahgal,3.44,27.24,19.24,...,6,Quant Mutual Fund,5,Equity,Sectoral / Thematic Mutual Funds,5.6,61.5,21.1,0.657207,2
2,Quant Tax Plan- Direct Growth,500,500,0.57,2779,10,Vasav Sahgal,3.50,17.63,19.74,...,6,Quant Mutual Fund,5,Equity,ELSS Mutual Funds,0.9,54.9,22.2,0.619385,3
3,Quant Multi Asset Fund,1000,5000,0.56,634,10,Vasav Sahgal,3.23,19.52,18.63,...,6,Quant Mutual Fund,5,Hybrid,Multi Asset Allocation Mutual Funds,10.6,42.6,20.5,0.590916,4
4,Quant Flexi Cap Fund,1000,5000,0.58,1044,10,Vasav Sahgal,3.65,15.93,19.16,...,6,Quant Mutual Fund,5,Equity,Flexi Cap Funds,6.8,47.7,18.2,0.581508,5


In [15]:
print(df.describe())

           min_sip  min_lumpsum  expense_ratio  fund_size_cr  fund_age_yr  \
count    30.000000    30.000000      30.000000     30.000000    30.000000   
mean    626.666667  3923.333333       0.551333   6264.966667     8.733333   
std     407.416870  1991.305815       0.246475   7518.172272     2.132399   
min       0.000000     0.000000       0.080000     89.000000     4.000000   
25%     150.000000  5000.000000       0.370000    971.750000     9.000000   
50%     750.000000  5000.000000       0.580000   3300.500000    10.000000   
75%    1000.000000  5000.000000       0.737500   8547.750000    10.000000   
max    1000.000000  5000.000000       1.000000  29953.000000    10.000000   

         sortino      alpha       beta     sharpe  risk_level     rating  \
count  30.000000  30.000000  30.000000  30.000000   30.000000  30.000000   
mean    4.068667  11.373333   0.869333   1.945333    5.766667   3.433333   
std     0.925965   5.019094   0.309013   0.199909    0.817200   2.028815   
mi

In [16]:
df['category'].unique()

array(['Equity', 'Hybrid', 'Other', 'Debt', 'Solution Oriented'],
      dtype=object)

In [18]:
df.isnull().sum()

scheme_name        0
min_sip            0
min_lumpsum        0
expense_ratio      0
fund_size_cr       0
fund_age_yr        0
fund_manager       0
sortino            0
alpha              0
sd                 0
beta               0
sharpe             0
risk_level         0
amc_name           0
rating             0
category           0
sub_category       0
returns_1yr        0
returns_3yr        0
returns_5yr        0
composite_score    0
rank               0
dtype: int64

## Data Cleaning

In [ ]:
# Data is already is clean but i have to show some cleaning process that why i show false cleaning for see the process
# replace missing value in return_3yr and return_5yr
mean_returns_3yr = df['returns_3yr'].mean()
mean_returns_5yr = df['returns_5yr'].mean()
df['returns_3yr'].fillna(mean_returns_3yr, inplace=True)
df['returns_5yr'].fillna(mean_returns_5yr, inplace=True)
print(df.isnull().sum())

scheme_name        0
min_sip            0
min_lumpsum        0
expense_ratio      0
fund_size_cr       0
fund_age_yr        0
fund_manager       0
sortino            0
alpha              0
sd                 0
beta               0
sharpe             0
risk_level         0
amc_name           0
rating             0
category           0
sub_category       0
returns_1yr        0
returns_3yr        0
returns_5yr        0
composite_score    0
rank               0
dtype: int64


C:\Users\HARSH PANDAY\AppData\Local\Temp\ipykernel_39000\2421392636.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['returns_3yr'].fillna(mean_returns_3yr, inplace=True)
C:\Users\HARSH PANDAY\AppData\Local\Temp\ipykernel_39000\2421392636.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behav

## Data Normalisation

In [21]:
from sklearn.preprocessing import MinMaxScaler

columns_to_normalize = ['expense_ratio', 'returns_1yr', 'returns_3yr', 'returns_5yr', 
                        'sharpe', 'sortino', 'alpha', 'beta']
df[columns_to_normalize] = df[columns_to_normalize].replace('-', pd.NA).apply(pd.to_numeric)

scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df[columns_to_normalize]), columns=columns_to_normalize)

# Adjust metrics where lower is better
df_normalized['expense_ratio'] = 1 - df_normalized['expense_ratio']
df_normalized['beta'] = 1 - df_normalized['beta']


## Composite Scoring

In [22]:
weights = {
    'expense_ratio': 0.2,
    'returns_1yr': 0.15,
    'returns_3yr': 0.15,
    'returns_5yr': 0.15,
    'sharpe': 0.1,
    'sortino': 0.1,
    'alpha': 0.1,
    'beta': 0.05
}

df_normalized['composite_score'] = sum(
    df_normalized[col] * weight for col, weight in weights.items()
)
df['composite_score'] = df_normalized['composite_score']


## Ranking Fund

In [23]:
df['rank'] = df['composite_score'].rank(ascending=False)
df_sorted = df.sort_values(by='rank')

## Exporting Result

In [24]:
df_top_30 = df_sorted.head(30)
df_top_30.to_excel('top_30_mutual_funds.xlsx', index=False)
print("Exported top 30 mutual funds to 'top_30_mutual_funds.xlsx'.")

Exported top 30 mutual funds to 'top_30_mutual_funds.xlsx'.
